In [1]:
import gc
import torch

import transformers
from datasets import load_dataset
from peft import LoraConfig
from trl import DPOTrainer, DPOConfig, SFTConfig, SFTTrainer
import bitsandbytes as bnb

from tqdm import tqdm
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024
lora_rank = 16

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.4, 
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth",
    random_state = 2811,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-12 20:59:36 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.4.1: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.8.4.
   \\   /|    NVIDIA GeForce RTX 4070 Ti SUPER. Num GPUs = 1. Max memory: 15.992 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
# print the first tensors parameters
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        break

base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight tensor([[ 0.0188,  0.0008,  0.0067,  ..., -0.0148,  0.0083, -0.0153],
        [ 0.0084, -0.0100, -0.0185,  ..., -0.0154, -0.0217, -0.0120],
        [-0.0092, -0.0051, -0.0143,  ...,  0.0147, -0.0104, -0.0125],
        ...,
        [ 0.0129,  0.0066, -0.0210,  ...,  0.0044,  0.0201, -0.0159],
        [-0.0025,  0.0097,  0.0134,  ...,  0.0007, -0.0125,  0.0168],
        [-0.0126, -0.0099,  0.0134,  ...,  0.0043, -0.0130, -0.0151]],
       device='cuda:0')


In [3]:
dataset = load_dataset("thainq107/Vi-Alpaca-Preference")
dataset, dataset["train"][0]

(DatasetDict({
     train: Dataset({
         features: ['id', 'question', 'chosen', 'rejected'],
         num_rows: 65017
     })
     test: Dataset({
         features: ['id', 'question', 'chosen', 'rejected'],
         num_rows: 2000
     })
 }),
 {'id': 'alpaca-7294',
  'question': 'Xác định và sửa lỗi ngữ pháp.\n\nTôi đã đi đến cửa hàng.',
  'chosen': 'Không có lỗi ngữ pháp. Câu này đã chính xác.',
  'rejected': 'Câu này không có lỗi ngữ pháp.'})

In [6]:
dataset["train"][1000]

{'id': 'alpaca-14472',
 'question': 'Tạo một meme bằng cách sử dụng cụm từ "thời trang muộn".',
 'chosen': '[Thêm hình ảnh người nổi tiếng mặc đồ lỗi thời, nhìn chẳng đẹp mắt, phù hợp với ý tưởng của meme] \n\nHãy tránh thời trang muộn, vì đôi khi nó có thể là điều tồi tệ nhất mà bạn có thể làm cho bản thân mình.',
 'rejected': 'Xin lỗi, nhưng vì tôi là một trí tuệ nhân tạo và không có khả năng thực hiện các tác vụ đồ họa, tôi không thể tạo ra một thiết kế áo phông cho bạn. Tuy nhiên, bạn có thể tham khảo các trang web thiết kế áo phông trực tuyến như Custom Ink, Printful hoặc Teespring để tạo ra thiết kế áo phông của riêng bạn dựa trên tiểu bang hoặc quốc gia của bạn. Chúc may mắn!'}

## **SFT**

In [5]:
def formatting_prompt_with_chat_template(example):
    conversation = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": example["question"]},
        {"role": "assistant", "content": example["chosen"]},
    ]
    prompt = tokenizer.apply_chat_template(
        conversation, tokenize=False, add_generation_prompt=False
    )
    return prompt

In [ ]:
hyperparameters = {
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 8,
    "gradient_checkpointing": True,
    "learning_rate": 3e-5,
    "logging_steps": 200,
    "num_train_epochs": 1,
    "save_strategy": "no",
    "overwrite_output_dir": True,
    "optim": "paged_adamw_8bit",
    "warmup_steps": 200,
    "bf16": True,
}
MAX_LENGTH = 1024

In [7]:
import os
os.environ['UNSLOTH_RETURN_LOGITS'] = '1'

In [4]:
SFT_OUTPUT_DIR = f"Qwen2.5-3B-Instruct-sft"

In [ ]:
sft_config = SFTConfig(
    **{ **hyperparameters, "output_dir":
       SFT_OUTPUT_DIR , "max_seq_length": MAX_LENGTH}
)


sft_trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=sft_config,
    train_dataset=dataset['train'],
    formatting_func=formatting_prompt_with_chat_template,
)

sft_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 65,017 | Num Epochs = 1 | Total steps = 4,063
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 29,933,568/3,000,000,000 (1.00% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: tridungluong123 (tridungluong123-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
200,1.559500
400,1.277200
600,1.250700
800,1.224200
1000,1.239700
1200,1.240200
1400,1.223200
1600,1.203500
1800,1.206600
2000,1.207000


TrainOutput(global_step=4063, training_loss=1.2372856487524853, metrics={'train_runtime': 12201.5926, 'train_samples_per_second': 5.329, 'train_steps_per_second': 0.333, 'total_flos': 3.3485348060494234e+17, 'train_loss': 1.2372856487524853})

In [ ]:
sft_trainer.push_to_hub("Qwen2.5-3B-instruct-sft-vi-alpaca", token="###")

wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


training_args.bin:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/120M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/binhphap5/Qwen2.5-3B-Instruct-sft/commit/68e5dc2edf2a5c0d24e828e7dc2f584146c1b8b4', commit_message='Qwen2.5-3B-instruct-sft-vi-alpaca', commit_description='', oid='68e5dc2edf2a5c0d24e828e7dc2f584146c1b8b4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/binhphap5/Qwen2.5-3B-Instruct-sft', endpoint='https://huggingface.co', repo_type='model', repo_id='binhphap5/Qwen2.5-3B-Instruct-sft'), pr_revision=None, pr_num=None)

## **DPO**

In [5]:
def convert_to_conversational_preference_format(example):
    return {
        "id": example["id"],
        "prompt": [{"role": "system",
                    "content": "You are a helpful assistant."},
                   {"role": "user",
                    "content": example["question"]}],
        "chosen": [{"role": "assistant",
                    "content": example["chosen"]}],
        "rejected": [{"role": "assistant",
                      "content": example["rejected"]}],
    }

dpo_dataset = dataset.map(convert_to_conversational_preference_format)


In [6]:
model.load_adapter(SFT_OUTPUT_DIR, is_trainable=True, adapter_name="dpo_full_adapter")

In [7]:
# print the first tensors parameters
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        break

model.layers.0.self_attn.q_proj.lora_A.dpo_full_adapter.weight tensor([[ 0.0251,  0.0029,  0.0091,  ..., -0.0146,  0.0087, -0.0091],
        [ 0.0020, -0.0121, -0.0184,  ..., -0.0148, -0.0165, -0.0157],
        [-0.0021, -0.0035, -0.0135,  ...,  0.0136, -0.0131, -0.0096],
        ...,
        [ 0.0069,  0.0047, -0.0242,  ...,  0.0051,  0.0216, -0.0196],
        [-0.0072,  0.0097,  0.0094,  ...,  0.0033, -0.0111,  0.0134],
        [-0.0059, -0.0047,  0.0145,  ...,  0.0026, -0.0188, -0.0106]],
       device='cuda:0')


In [14]:
def get_model_response(model, tokenizer, instruction):
    cur_conversation = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": instruction}
    ]
    cur_input_prompt = tokenizer.apply_chat_template(
        cur_conversation, add_generation_prompt=True, tokenize=True
    )
    cur_output_ids = model.generate(
        input_ids=torch.LongTensor([cur_input_prompt]).to(model.device),
        attention_mask=torch.ones(1, len(cur_input_prompt)).to(model.device),
        max_new_tokens=1000,
        pad_token_id=tokenizer.eos_token_id,
    )
    cur_generated_ids = cur_output_ids[0][len(cur_input_prompt):]
    return tokenizer.decode(cur_generated_ids, skip_special_tokens=True)

instruction = "Tạo một meme bằng cách sử dụng cụm từ 'thời trang muộn'."
response = get_model_response(model, tokenizer, instruction)
print(f"Instruction: {instruction}")
print(f"Response: {response}")

Instruction: Tạo một meme bằng cách sử dụng cụm từ 'thời trang muộn'.
Response: Đây là một meme với tiêu đề "Thời trang muộn" và hình ảnh của một người đàn ông đang mặc một chiếc áo sơ mi màu đen, quần jeans và giày cao gót.

Meme:

Thời trang muộn

Người đàn ông này đã quá muộn để đi mua quần áo mới cho mình. Anh ta đang mặc một chiếc áo sơ mi màu đen và quần jeans cũ kỹ, nhưng vẫn còn cố gắng để trông sang trọng hơn bằng cách đội một chiếc mũ cao cấp và đeo một chiếc túi xách đắt tiền.

Vậy là anh ta đã trở thành một trong những người đàn ông thời trang muộn nhất trên đường phố.


In [16]:
dpo_dataset['train'][1000]

{'id': 'alpaca-14472',
 'question': 'Tạo một meme bằng cách sử dụng cụm từ "thời trang muộn".',
 'chosen': [{'content': '[Thêm hình ảnh người nổi tiếng mặc đồ lỗi thời, nhìn chẳng đẹp mắt, phù hợp với ý tưởng của meme] \n\nHãy tránh thời trang muộn, vì đôi khi nó có thể là điều tồi tệ nhất mà bạn có thể làm cho bản thân mình.',
   'role': 'assistant'}],
 'rejected': [{'content': 'Xin lỗi, nhưng vì tôi là một trí tuệ nhân tạo và không có khả năng thực hiện các tác vụ đồ họa, tôi không thể tạo ra một thiết kế áo phông cho bạn. Tuy nhiên, bạn có thể tham khảo các trang web thiết kế áo phông trực tuyến như Custom Ink, Printful hoặc Teespring để tạo ra thiết kế áo phông của riêng bạn dựa trên tiểu bang hoặc quốc gia của bạn. Chúc may mắn!',
   'role': 'assistant'}],
 'prompt': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'Tạo một meme bằng cách sử dụng cụm từ "thời trang muộn".',
   'role': 'user'}]}

In [17]:
hyperparameters = {
    "per_device_train_batch_size": 3,
    "gradient_accumulation_steps": 8,
    "gradient_checkpointing": True,
    "learning_rate": 3e-5,
    "logging_steps": 100,
    "max_steps": 500,
    "save_strategy": "no",
    "overwrite_output_dir": True,
    "optim": "paged_adamw_8bit",
    "warmup_steps": 20,
    "bf16": True,
}
MAX_LENGTH = 512

In [18]:
# Use wandb
import wandb
wandb.init(
    project="vi-alpaca-preference",
    name="Qwen2.5-3B-4bit-dpo"
)


wandb: Currently logged in as: tridungluong123 (tridungluong123-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [19]:
DPO_OUTPUT_DIR = f"Qwen2.5-3B-Instruct-dpo"
dpo_args = DPOConfig(
    **{ **hyperparameters, "output_dir":
       DPO_OUTPUT_DIR, "max_length": MAX_LENGTH }
)

dpo_trainer = DPOTrainer(
    model,
    args=dpo_args,
    train_dataset=dpo_dataset['train'],
    processing_class=tokenizer,
)
dpo_trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 65,017 | Num Epochs = 1 | Total steps = 500
O^O/ \_/ \    Batch size per device = 3 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (3 x 8 x 1) = 24
 "-____-"     Trainable parameters = 29,933,568/3,000,000,000 (1.00% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
100,0.300700
200,0.161900
300,0.181500
400,0.189400
500,0.157200


TrainOutput(global_step=500, training_loss=0.19813651466369628, metrics={'train_runtime': 6097.8911, 'train_samples_per_second': 1.968, 'train_steps_per_second': 0.082, 'total_flos': 0.0, 'train_loss': 0.19813651466369628, 'epoch': 0.18456143588797122})

In [75]:
torch.cuda.empty_cache()

In [83]:
instruction = "Tạo một meme bằng cách sử dụng cụm từ 'thời trang muộn'."
response = get_model_response(model, tokenizer, instruction)
print(f"Instruction: {instruction}")
print(f"Response: {response}")

Instruction: Tạo một meme bằng cách sử dụng cụm từ 'thời trang muộn'.
Response: [Image: A person wearing outdated and mismatched clothes, with the caption "Thời trang muộn"]

Explanation: The meme uses the phrase "thời trang muộn" which means "late fashion" or "outdated fashion". The image shows a person wearing clothes that are not in trend, creating a meme that satirizes those who choose to wear outdated fashion styles.


In [71]:
dpo_trainer.save_model(DPO_OUTPUT_DIR)

In [ ]:
dpo_trainer.push_to_hub("Qwen2.5-3B-instruct-dpo-vi-alpaca", token="###")

wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


adapter_model.safetensors:   0%|          | 0.00/116M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/6.20k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/binhphap5/Qwen2.5-3B-Instruct-dpo/commit/9d37307b02ae59c86cac29fb998fb000780c97b9', commit_message='Qwen2.5-3B-instruct-dpo-vi-alpaca', commit_description='', oid='9d37307b02ae59c86cac29fb998fb000780c97b9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/binhphap5/Qwen2.5-3B-Instruct-dpo', endpoint='https://huggingface.co', repo_type='model', repo_id='binhphap5/Qwen2.5-3B-Instruct-dpo'), pr_revision=None, pr_num=None)